In [1]:
import os
import glob
import cv2
import numpy as np
import zipfile

In [2]:
daisy_dir = os.path.join('/content/drive/MyDrive/flowers/daisy')
dandelion_dir = os.path.join('/content/drive/MyDrive/flowers/dandelion')
rose_dir = os.path.join('/content/drive/MyDrive/flowers/rose')
sunflower_dir = os.path.join('/content/drive/MyDrive/flowers/sunflower')
tulip_dir = os.path.join('/content/drive/MyDrive/flowers/tulip')

print('total training daisy images:', len(os.listdir(daisy_dir)))
print('total training dandelion images:', len(os.listdir(dandelion_dir)))
print('total training rose images:', len(os.listdir(rose_dir)))
print('total training sunflower images:', len(os.listdir(sunflower_dir)))
print('total training tulip images:', len(os.listdir(tulip_dir)))

total training daisy images: 769
total training dandelion images: 1055
total training rose images: 784
total training sunflower images: 734
total training tulip images: 984


In [3]:
def read_images(directory):
  for img in glob.glob(directory + "/*.jpg"):
    image = cv2.imread(img)
    resized_image = cv2.resize(image/255, (240, 320))
    
    yield resized_image

In [4]:
daisy_resized_imgs = np.array(list(read_images(daisy_dir)))

In [5]:
rose_resized_imgs = np.array(list(read_images(rose_dir)))

In [6]:
sunflower_resized_imgs = np.array(list(read_images(sunflower_dir)))

In [7]:
tulip_resized_imgs = np.array(list(read_images(tulip_dir)))

In [8]:
dandelion_resized_imgs = np.array(list(read_images(dandelion_dir)))

In [ ]:
daisy_labels = np.ones((daisy_resized_imgs.shape[0],1))
dandelion_labels = np.ones((dandelion_resized_imgs.shape[0],1))*2
rose_labels = np.ones((rose_resized_imgs.shape[0],1))*3
sunflower_labels = np.ones((sunflower_resized_imgs.shape[0],1))*4
tulip_labels = np.ones((tulip_resized_imgs.shape[0],1))*5

In [ ]:
all_flowers = np.concatenate((daisy_resized_imgs,dandelion_resized_imgs,rose_resized_imgs,sunflower_resized_imgs,tulip_resized_imgs)) 

In [ ]:
all_flowers_labels = np.concatenate((daisy_labels,dandelion_labels,rose_labels,sunflower_labels,tulip_labels))

In [ ]:
all_flowers_labels.shape

(4323, 1)

In [ ]:
all_flowers_falttened = all_flowers.reshape(len(all_flowers), 320*240*3)

In [ ]:
from keras.utils import to_categorical
all_flowers_labels_encoded = to_categorical(all_flowers_labels)

In [ ]:
all_flowers_labels_encoded

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
np.random.seed(1234)
mask = np.random.rand(len(all_flowers_falttened)) < 0.6
train_data = all_flowers_falttened[mask]
test_data = all_flowers_falttened[~mask]
train_label = all_flowers_labels_encoded[mask]
test_label = all_flowers_labels_encoded[~mask]

In [ ]:
from keras import models
from keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(230400,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(46, activation='softmax'))
  model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
# Readjusting epochs
history = model.fit(train_data, train_label, epochs=200, validation_split=0.2, verbose=0)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
evaluate = model.evaluate(test_data, test_label, verbose=0)

In [ ]:
evaluate

In [ ]:
test_predictions = model.predict(test_data)

In [ ]:
test_predictions[:10]

In [ ]:
test_labels[:10]